In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [2]:
import pandas as pd

# Importing data

In [3]:
raw_data = pd.read_csv("train.csv")

In [4]:
clean_data = raw_data.drop(["Cabin", "Name", "PassengerId", "Ticket"], axis=1)

In [5]:
clean_data = pd.get_dummies(clean_data).fillna(-1)

In [9]:
train, test = train_test_split(clean_data)

In [10]:
def split_x_y(dataframe, target):
    return dataframe.drop(target, axis=1), dataframe[target]

In [11]:
X_train, y_train = split_x_y(train, "Survived")
X_test, y_test = split_x_y(test, "Survived")

# Baseline model

In [21]:
rf = RandomForestClassifier(n_estimators=100, n_jobs=-1)
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=-1, oob_score=False,
            random_state=None, verbose=0, warm_start=False)

In [22]:
y_pred = rf.predict_proba(X_test)

In [23]:
auc = roc_auc_score(y_true=y_test, y_score=y_pred[:, 1])

In [24]:
auc

0.83213147744423954

# Deep Forest

In [26]:
from sklearn.model_selection import StratifiedKFold

In [27]:
rf1 = RandomForestClassifier(n_estimators=100, n_jobs=-1)
rf2 = RandomForestClassifier(n_estimators=200, n_jobs=-1)

In [44]:
rf1.fit(X_train, y_train)
y_pred_train_1 = rf1.predict_proba(X_train)
y_pred_1 = rf1.predict_proba(X_test)

y_pred_train_1 = pd.DataFrame(y_pred_train_1, columns=["rf1_0", "rf1_1"], index=X_train.index)

In [45]:
rf2.fit(X_train, y_train)
y_pred_train_2 = rf2.predict_proba(X_train)
y_pred_2 = rf2.predict_proba(X_test)

y_pred_train_2 = pd.DataFrame(y_pred_train_2, columns=["rf2_0", "rf2_1"], index=X_train.index)

In [48]:
hidden_train_1 = pd.concat([X_train, y_pred_train_1, y_pred_train_2], axis=1)

In [49]:
hidden_train_1.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,rf1_0,rf1_1,rf2_0,rf2_1
414,3,44.0,0,0,7.9250,0,1,0,0,1,0.278333,0.721667,0.331667,0.668333
542,3,11.0,4,2,31.2750,1,0,0,0,1,1.000000,0.000000,0.995000,0.005000
42,3,-1.0,0,0,7.8958,0,1,1,0,0,0.930000,0.070000,0.925000,0.075000
284,1,-1.0,0,0,26.0000,0,1,0,0,1,0.770000,0.230000,0.755000,0.245000
621,1,42.0,1,0,52.5542,0,1,0,0,1,0.290000,0.710000,0.250000,0.750000
